In [2]:
import os
import sys
import numpy as np
sys.path.append('/home/eddyod/programming/activebrainatlas')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()

In [3]:
from neuroglancer.models import AnnotationSession, BrainRegion
from django.contrib.auth.models import User

In [4]:
from neuroglancer.models import MarkedCell


def get_session1(annotator, animal, brain_region, annotation_type, cell_type, source):
    session = None
    sessions = AnnotationSession.objects.filter(animal=animal)\
                        .filter(brain_region=brain_region)\
                        .filter(annotator=annotator)\
                        .filter(annotation_type=annotation_type)\
                        .filter(active=True).all()
    if annotation_type == 'MARKED_CELL':
        cell_types = []
        sources = []
        ids = []
        for i, session in enumerate(sessions):
            # print(f'session ID={session.id}')
            if session.cell_type is not None and session.source is not None:
                cell_types.append(session.cell_type.cell_type)
                sources.append(session.source)
                ids.append(i)
        #print(sources)
        if cell_types ==[]:
            print('cell types is empty')
        right_cell_type_and_source = \
            np.logical_and(np.array(cell_types)==cell_type, np.array(sources)==source)
        #print(right_cell_type_and_source)
        found = sum(right_cell_type_and_source)
        assert found<=1
        if found==0:
            session = None
        else:
            session = sessions[ids[int(np.where(right_cell_type_and_source)[0][0])]]
    else:
        assert len(sessions)==1
        session = sessions[0]
    return session

def get_session2(annotator, animal, brain_region, annotation_type, cell_type, source):
    session = None
    sessions = AnnotationSession.objects.filter(animal=animal)\
                        .filter(brain_region=brain_region)\
                        .filter(annotator=annotator)\
                        .filter(annotation_type=annotation_type)\
                        .filter(active=True).all()
    if annotation_type == 'MARKED_CELL':
        marked_cells = MarkedCell.objects.filter(annotation_session__animal=animal)\
                            .filter(annotation_session__brain_region=brain_region)\
                            .filter(annotation_session__annotator=annotator)\
                            .filter(annotation_session__annotation_type=annotation_type)\
                            .filter(annotation_session__active=True)\
                            .filter(cell_type__cell_type=cell_type)\
                            .all()
        if source is not None:
            marked_cells = marked_cells.filter(source=source)
        marked_cell = marked_cells.first()
        session = marked_cell.annotation_session
        #print(marked_cells.query)
    else:
        assert len(sessions)==1
        session = sessions.first()
    return session


In [5]:
annotator = User.objects.get(pk=38)
animal = 'DK41'
brain_region = BrainRegion.objects.get(pk=52)
annotation_type = 'MARKED_CELL'
cell_type = 'mixed'
source = 'HUMAN_NEGATIVE'
session = get_session1(annotator, animal, brain_region, annotation_type, cell_type, source)
print('version 1', session)
if session is not None:
    print(session.id)
session = get_session2(annotator, animal, brain_region, annotation_type, cell_type, source)
print('version 2', session)
if session is not None:
    print(session.id)

version 1 DK41 Point from annotation layer point MARKED_CELL
6393
version 2 DK41 Point from annotation layer point MARKED_CELL
6393
